In [ ]:
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.memory import ConversationBufferMemory
from langchain.tools.wikipedia.tool import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.llms import HuggingFacePipeline

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

class ResearchAgent:
    def __init__(self, llm):
        self.llm = llm
        self.memory = ConversationBufferMemory(memory_key="chat_history")
        
        # Define tools
        self.tools = [
            Tool(
                name="Wikipedia",
                func=self.search_wikipedia,
                description="Useful for searching Wikipedia articles"
            ),
            Tool(
                name="Calculator",
                func=self.calculate,
                description="Useful for performing mathematical calculations"
            )
        ]
        
        # Initialize agent
        self.agent = initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
            memory=self.memory,
            verbose=True
        )
    
    def search_wikipedia(self, query):
        # Initialize Wikipedia wrapper and query tool
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)
    
    def calculate(self, expression):
        try:
            # This is dangerous; replace with safer parsing for production
            result = eval(expression)
            return str(result)
        except Exception as e:
            return f"Error in calculation: {str(e)}"
    
    def run(self, query):
        return self.agent.run(query)


# ---------- Usage Example ----------
# Load your model
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

# Hugging Face pipeline
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() or torch.backends.mps.is_available() else -1,
    max_length=512,
    temperature=0
)

# Wrap the pipeline in HuggingFacePipeline for LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Initialize ResearchAgent
research_agent = ResearchAgent(llm=llm)

# Run a query
result = research_agent.run(
    "Tell me about artificial intelligence and calculate how many years since it was first coined as a term in 1956."
)

print(result)

In [ ]:
import sys
sys.path.append('/Users/tkang/Documents/Projects/MDS/llm-based-workflow-for-text-classification-and-translation')

import os
from dotenv import load_dotenv
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain import HuggingFacePipeline

from src.milestone2.models.llm import LLM
from src.milestone2.models.agent import Agent

load_dotenv()

# llm = LLM().create("ollama-llama3.2")
llm = LLM().create("medium")
# wrap it in hugging face pipeline
llm = HuggingFacePipeline(pipeline=llm)

agent = Agent(llm)
result = agent.run({"input": "Ugh I'm so frustrated"})
print(result)

# sent_subagent = SentimentAnalysisSubAgent()
# res = sent_subagent.analyze(llm, "I am happy")
# print(res)

In [ ]:
### sentiment analysis
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import gc

class SentimentAnalysisSubAgent:
    def __init__(self):
        model_id = "ibm-granite/granite-3.0-2b-instruct"

        self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        self.model = AutoModelForCausalLM.from_pretrained(model_id)

        # Use MPS (Apple Silicon)
        self.device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
        self.model = self.model.to(self.device)

        self.generator = pipeline(
            "text-generation",  # <-- Use "text-generation" for CausalLM models
            model=self.model,
            tokenizer=self.tokenizer,
            return_full_text=False,
            max_length=512,
            temperature=0,
            device=-1  # Explicitly use CPU/MPS for pipeline (Hugging Face currently doesn't fully support MPS in `pipeline`)
        )

        self.prompt_description = """
        You are a sentiment analysis expert. Analyze the following sentence and return:
        1. The overall sentiment: Positive, Negative, Neutral, or Mixed.
        2. An explanation describing why it was classified that way.
        """

        self.prompt_output_format = """
        Response Format (JSON):
        {
            "sentiment_label": "...",
            "explanation": "..."
        }
        """

    def analyze(self, question):
        full_prompt = (
            self.prompt_description +
            f"\nSentence: {question}\n" +
            self.prompt_output_format
        )

        result = self.generator(full_prompt, max_length=512, do_sample=False)

        # If you want to free memory after one inference
        # self.free_mps_memory()

        return result
    
    # def free_mps_memory(self):
    #     del self.model
    #     del self.tokenizer
    #     gc.collect()

    #     # Only if on MPS and using PyTorch >= 2.0
    #     if torch.backends.mps.is_available() and hasattr(torch.mps, "empty_cache"):
    #         torch.mps.empty_cache()

In [ ]:
sentiment_analysis_agent = SentimentAnalysisSubAgent()

res = sentiment_analysis_agent.analyze("I love this product!")
print(res)

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load DetoxLLM-7B model and tokenizer from HuggingFace
model_name = "UBC-NLP/DetoxLLM-7B"

# If you have a GPU, use device_map="auto". For CPU-only, add torch_dtype=torch.float32
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Create a text generation pipeline
detoxify_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=256,
    do_sample=True,
    temperature=0.1,
    top_k=50,
    top_p=0.95
)

# Detoxify function
def detoxify_sentence(text: str) -> str:
    prompt = (
        "### Instruction:\n"
        "Detoxify the following sentence while keeping its meaning intact.\n\n"
        f"### Input:\n{text}\n\n### Response:"
    )
    response = detoxify_pipeline(prompt, num_return_sequences=1)
    detoxified_text = response[0]['generated_text'].split("### Response:")[-1].strip()
    return detoxified_text

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


In [3]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from textblob import TextBlob
from detoxify import Detoxify
import openai
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# model_name = "mistralai/Mistral-7B-v0.1"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto")
# generation_pipeline = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_length=512,
#     temperature=0.1,
#     # top_p=0.95,
#     # repetition_penalty=1.2
# )

# llm = HuggingFacePipeline(pipeline=generation_pipeline)
gpt_llm = ChatOpenAI(model_name="gpt-4", temperature=0)

def sentiment_analysis_with_reason(text: str) -> str:
    prompt = (
        f"What is the sentiment (Positive, Negative, Neutral) of the following sentence? "
        f"Explain your reasoning in detail.\n\n"
        f"Sentence: \"{text}\""
    )
    
    # This uses the agent's LLM to answer the question (chat interface)
    agent_llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    response = agent_llm.predict(prompt)
    return response

# --- Toxicity Analysis with Reasoning Tool ---
def toxicity_analysis_with_reason(text: str) -> str:
    prompt = (
        f"Is the following sentence toxic or non-toxic? "
        f"Explain your reasoning in detail.\n\n"
        f"Sentence: \"{text}\""
    )
    
    # This uses the agent's LLM to answer the question (chat interface)
    agent_llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    response = agent_llm.predict(prompt)
    return response

# --- Translate to English Tool ---
def translate_to_english(text: str) -> str:
    prompt = f"Translate the following sentence into English:\n\n{text}"
    
    # This uses the agent's LLM to answer the question (chat interface)
    agent_llm = ChatOpenAI(model_name="gpt-4", temperature=0)
    response = agent_llm.predict(prompt)
    return response

# --- Create Tools ---
tools = [
    Tool.from_function(
        func=translate_to_english,
        name="TranslateToEnglish",
        description="Use this tool to translate sentences into English."
    ),
    Tool.from_function(
        func=sentiment_analysis_with_reason,
        name="SentimentAnalysisWithReason",
        description="Use this tool to analyze sentiment of a sentence. Returns sentiment label and reasoning."
    ),
    Tool.from_function(
        func=toxicity_analysis_with_reason,
        name="ToxicityAnalysisWithReason",
        description="Use this tool to analyze toxicity of a sentence. Returns toxicity label and reasoning."
    ),
    Tool.from_function(
        func=detoxify_sentence,
        name="DetoxifySentence",
        description="Use this tool to detoxify a toxic sentence and rewrite it without harmful language."
    )
]

# --- Initialize LangChain Agent ---

agent = initialize_agent(
    tools=tools,
    llm=gpt_llm,
    # llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example usage
if __name__ == "__main__":
    query = (
        "Here's a sentence: 'You're so stupid and annoying! I can't stand you.'\n"
        "Please translate it to English if necessary, analyze its sentiment with reasons, "
        "analyze its toxicity with reasons, and detoxify it.\n"
        "Return the result strictly as a JSON array in this format:\n\n"
        "[\n"
        "    {\n"
        "        \"sentiment_label\": \"<Sentiment Label>\",\n"
        "        \"explanation\": \"<Explain why the sentiment was classified as such>\"\n"
        "    },\n"
        "    {\n"
        "        \"toxicity_label\": \"<Toxic/Non-Toxic>\",\n"
        "        \"explanation\": \"<Explain why the sentence was considered toxic or not>\"\n"
        "    },\n"
        "    {\n"
        "        \"detoxified_text\": \"<Detoxified Sentence>\",\n"
        "        \"explanation\": \"<Explain how you modified the sentence to make it non-toxic>\"\n"
        "    }\n"
        "]"
    )

    response = agent.run(query)

    print("\nAgent Response:\n")
    print(response)

    # while True:
    #     query = input("\nAsk the agent to perform a task (or type 'exit'): ")
    #     if query.lower() == "exit":
    #         break
    #     response = agent.run(query)
    #     print(f"\nAgent Response: {response}")

/var/folders/8b/lb1cxfqj6zx5dtq789h2k8xr0000gn/T/ipykernel_28356/2937384922.py:87: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/var/folders/8b/lb1cxfqj6zx5dtq789h2k8xr0000gn/T/ipykernel_28356/2937384922.py:118: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run



> Entering new AgentExecutor chain...
The sentence is already in English, so no translation is needed. I need to analyze its sentiment and toxicity, and then detoxify it.
Action: SentimentAnalysisWithReason
Action Input: 'You're so stupid and annoying! I can't stand you.'

/var/folders/8b/lb1cxfqj6zx5dtq789h2k8xr0000gn/T/ipykernel_28356/2937384922.py:36: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_llm.predict(prompt)



Observation: The sentiment of the sentence is Negative. The reasoning behind this is that the speaker is using derogatory language such as "stupid" and "annoying" to describe the person they are speaking to. Additionally, the phrase "I can't stand you" is a strong expression of dislike or irritation. All these elements contribute to a negative sentiment.
Thought:Now that I have analyzed the sentiment of the sentence, I need to analyze its toxicity.
Action: ToxicityAnalysisWithReason
Action Input: 'You're so stupid and annoying! I can't stand you.'
Observation: The sentence is toxic. It contains direct insults and negative language aimed at another person, which is a form of verbal abuse. The speaker is expressing their dislike and intolerance towards the person they are addressing in a disrespectful and harmful manner. This kind of communication can be damaging to the recipient's mental and emotional well-being.
Thought:

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Now that I have analyzed the toxicity of the sentence, I need to detoxify it.
Action: DetoxifySentence
Action Input: 'You're so stupid and annoying! I can't stand you.'
Observation: You're not very intelligent and can be bothersome. I find you quite dislikable.
Thought:I now know the final answer
Final Answer: 
[
    {
        "sentiment_label": "Negative",
        "explanation": "The sentiment of the sentence is Negative. The reasoning behind this is that the speaker is using derogatory language such as 'stupid' and 'annoying' to describe the person they are speaking to. Additionally, the phrase 'I can't stand you' is a strong expression of dislike or irritation. All these elements contribute to a negative sentiment."
    },
    {
        "toxicity_label": "Toxic",
        "explanation": "The sentence is toxic. It contains direct insults and negative language aimed at another person, which is a form of verbal abuse. The speaker is expressing their dislike and intolerance towards the p